In [1]:
import pandas as pd

In [2]:
splits = {'train': 'wiki.full.aner.ori.train.95.tsv', 'validation': 'wiki.full.aner.ori.valid.95.tsv', 'test': 'wiki.full.aner.ori.test.95.tsv'}
train = pd.read_csv("hf://datasets/bogdancazan/wikilarge-text-simplification/" + splits["train"], sep="\t")
val = pd.read_csv("hf://datasets/bogdancazan/wikilarge-text-simplification/" + splits["validation"], sep="\t")
test = pd.read_csv("hf://datasets/bogdancazan/wikilarge-text-simplification/" + splits["test"], sep="\t")